In [10]:
import numpy as np
from scflp_v2 import SCFLPData, SCFLPSolver, make_random_instance

In [11]:
I = 60
J = 60
p = 2
r = 2
beta = 0.1

In [12]:
seed = np.random.SeedSequence()
# 1) Build a toy instance (no pre-existing facilities)
data = make_random_instance(I=I, J=J, p=p, r=r, beta=beta, seed=seed)

# 2) Solve (PuLP/CBC by default). If you have Gurobi, use milp_backend='gurobi'
solver = SCFLPSolver(data, milp_backend="pulp", pulp_solver="CBC")
result = solver.solve(max_rounds=2000, verbose=True)

════════════════════════════════════════════════════════════════════════════════
🚀 S-CFLP を解きます（DCG + Bulge/Submodular カット）
════════════════════════════════════════════════════════════════════════════════
➤ 候補地 J = 60, 需要点 I = 60, p = 2, r = 2
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 1: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [2, 6]（r = 2）。上界 α − βᵀŷ = 0.747450
➤ 📐 評価：L(x̂,ŷ) = 0.415102 → 違反量 \hat(θ) - L = 5.849e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 2: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [25, 57]（r = 2）。上界 α − βᵀŷ = 0.775564
➤ 📐 評価：L(x̂,ŷ) = 0.557500 → 違反量 \hat(θ) - L = 4.425e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 3: master 解 → \hat(θ) = 0.796821, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [2, 25]（r = 2）。上界 α − βᵀŷ = 0.764092
➤ 📐 評価：L(x̂,ŷ) = 0.575099

In [13]:
print("\n=== RESULT ===")
x = result["x"]
print("Objective (leader market share):", result["obj"])
print("Selected sites (indices where x=1):", np.where(x > 0.5)[0].tolist())
print(
    "theta:",
    result["theta"],
    "sum x:",
    x.sum(),
    "iterations:",
    result["iterations"],
    "cuts:",
    result["cuts"],
)



=== RESULT ===
Objective (leader market share): 0.58064715
Selected sites (indices where x=1): [6, 12]
theta: 0.58064715 sum x: 2.0 iterations: 12 cuts: 22


## 10回回して平均取る

In [14]:
import numpy as np

I, J, p, r, beta = 10, 10, 2, 2, 0.1  # 例: パラメータ
n_runs = 10
results = []

for run in range(n_runs):
    seed = np.random.SeedSequence()  # 各回ごとに新しい乱数シード
    data = make_random_instance(I=I, J=J, p=p, r=r, beta=beta, seed=seed)

    solver = SCFLPSolver(data, milp_backend="pulp", pulp_solver="CBC")
    result = solver.solve(max_rounds=2000, verbose=False)  # verboseを抑制
    results.append(result)

# --- 平均をとる ---
# resultが辞書形式を仮定し、数値部分を平均化
from collections import defaultdict

averages = defaultdict(float)
for res in results:
    for k, v in res.items():
        if isinstance(v, (int, float)):  # 数値のみ対象
            averages[k] += v / n_runs

print("平均結果:")
for k, v in averages.items():
    print(f"{k}: {v}")


════════════════════════════════════════════════════════════════════════════════
🚀 S-CFLP を解きます（DCG + Bulge/Submodular カット）
════════════════════════════════════════════════════════════════════════════════
➤ 候補地 J = 10, 需要点 I = 10, p = 2, r = 2
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 1: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [3, 5]（r = 2）。上界 α − βᵀŷ = 0.615497
➤ 📐 評価：L(x̂,ŷ) = 0.481040 → 違反量 \hat(θ) - L = 5.190e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 2: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [0, 2]（r = 2）。上界 α − βᵀŷ = 0.653631
➤ 📐 評価：L(x̂,ŷ) = 0.478742 → 違反量 \hat(θ) - L = 5.213e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 3: master 解 → \hat(θ) = 0.768923, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [0, 3]（r = 2）。上界 α − βᵀŷ = 0.656509
➤ 📐 評価：L(x̂,ŷ) = 0.548424 → 

## それぞれのインスタンスを平均してCSVに保存

In [17]:
import numpy as np
import pandas as pd
from collections import defaultdict

# パラメータの候補
instances = [
    {"I": 10, "J": 10, "p": 2, "r": 2, "beta": 0.1},
    {"I": 20, "J": 20, "p": 2, "r": 2, "beta": 0.1},
    {"I": 30, "J": 30, "p": 2, "r": 2, "beta": 0.1},
]

n_runs = 10
all_results = []

for inst_id, params in enumerate(instances, start=1):
    run_results = []
    for run in range(n_runs):
        seed = np.random.SeedSequence()
        data = make_random_instance(**params, seed=seed)
        
        solver = SCFLPSolver(data, milp_backend="pulp", pulp_solver="CBC")
        result = solver.solve(max_rounds=2000, verbose=False)
        
        run_results.append(result)

    # --- インスタンスごとに平均を計算 ---
    avg_theta = np.mean([res["theta"] for res in run_results])
    avg_time = np.mean([res["time_sec"] for res in run_results])

    row = {
        "I": params["I"],
        "J": params["J"],
        "p": params["p"],
        "r": params["r"],
        "beta": params["beta"],
        "theta": avg_theta,
        "time_sec": avg_time
    }
    all_results.append(row)

# --- DataFrameにまとめてCSV保存 ---
df = pd.DataFrame(all_results)
df.to_csv("scflp_results_avg.csv", index=False, encoding="utf-8")

print("保存完了: scflp_results_avg.csv")



════════════════════════════════════════════════════════════════════════════════
🚀 S-CFLP を解きます（DCG + Bulge/Submodular カット）
════════════════════════════════════════════════════════════════════════════════
➤ 候補地 J = 10, 需要点 I = 10, p = 2, r = 2
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 1: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [3, 4]（r = 2）。上界 α − βᵀŷ = 0.654566
➤ 📐 評価：L(x̂,ŷ) = 0.478919 → 違反量 \hat(θ) - L = 5.211e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 2: master 解 → \hat(θ) = 1.000000, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [1, 7]（r = 2）。上界 α − βᵀŷ = 0.666202
➤ 📐 評価：L(x̂,ŷ) = 0.536216 → 違反量 \hat(θ) - L = 4.638e-01
✂️  カット追加: BULGE + SUBMOD
────────────────────────────────────────────────────────────────────────────────
➤ 🧮 ラウンド 3: master 解 → \hat(θ) = 0.775234, |x| = 2/2（整数）
➤ 🔎 分離：ŷ は [0, 4]（r = 2）。上界 α − βᵀŷ = 0.645548
➤ 📐 評価：L(x̂,ŷ) = 0.516776 → 